In [ ]:
!pip install bitsandbytes==0.43.1
!pip install accelerate==0.30.1
# !pip install transformers==4.39.3
!pip install transformers==4.42.3
!pip install gradio==4.29.0

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
# Specify the model ID for the pre-trained model from Hugging Face
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# Load the tokenizer using the specified model ID
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Configure the BitsAndBytes settings for loading the model
config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=config
)

# Chain-of-Thought(CoT) Prompt Engineering
- LLM이 답변을 생성할 때, 단순히 최종 답만 생성하는 것이 아닌, 그 과정을 단계별로 나누어서 생각하도록 하는 기법
- 추론 과정의 투명성 : LLM의 작동방식을 더 잘 이해할 수 있음
- 복잡한 문제 해결 능력 : 여러 단계의 추론 과정을 효과적으로 모델링할 수 있음
- 학습 효율성 향상 : 새로운 문제에 대해 빠르게 적응할 수 있게
```
Input
질문 : 너는 리뷰의 긍부정을 알려주는 로봇이야
    왜 그렇게 생각했는지 말해주고, 그에 따른 분류를 해주면 돼.
    "오늘 시켰는데 정말 맛있어요"
대답 : "맛있어요" 키워드가 있으므로, "긍정"
질문 : "별로 였어요 ㅠ"
대답 : "별로" 키워드가 있으므로, "부정"
질문 : "한번 더 시켜먹고 싶어요"

Output
대답 : "한번 더 시켜먹고" 키워드가 있으므로 "긍정"
```

In [ ]:
# CoT는 시스템 프롬프트가 따로 없음.
# few-shot을 통해 응답값 조율
messages = [
    {"role": "user", "content": "Can you help me solve this quadratic equation: x^2 - 3x - 4 = 0?"},
    {"role": "assistant", "content": """
    Sure, to solve the quadratic equation x^2 - 3x - 4 = 0,
    we can use the quadratic formula, x = [-b ± sqrt(b² - 4ac)] / (2a).
    In this equation, a=1, b=-3, and c=-4.
    First, let's calculate the value under the square root,
    b² - 4ac, which is (-3)² - 4*1*(-4).
    The value is 9 + 16, which equals to 25. Next, we need to compute the values of x using the quadratic formula.
    So, x = [3 ± sqrt(25)] / 2. The roots of the equation are x = 4 and x = -1.
    """
    },

    {"role": "user", "content": "Can you help me solve this quadratic equation: x^2 - 6x + 9 = 0?"}
]

for message in messages:
    message["content"] = message["content"].replace("\n", "")

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.3,
    top_p=0.9
)

In [ ]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

# Zero-shot Prompt Engineering


In [ ]:
# 윤리적 필터링

messages = [{"role": "user", "content": """You're given a list of moderation categories as below:
- illegal: Illegal activity.
- child abuse: child sexual abuse material or any content that exploits or harms children.
- hate violence harassment: Generation of hateful, harassing, or violent content: content that expresses, incites, or promotes hate based on identity, content that intends to harass, threaten, or bully an individual, content that promotes or glorifies violence or celebrates the suffering or humiliation of others.
- malware: Generation of malware: content that attempts to generate code that is designed to disrupt, damage, or gain unauthorized access to a computer system.
- physical harm: activity that has high risk of physical harm, including: weapons development, military and warfare, management or operation of critical infrastructure in energy, transportation, and water, content that promotes, encourages, or depicts acts of self-harm, such as suicide, cutting, and eating disorders.
- economic harm: activity that has high risk of economic harm, including: multi-level marketing, gambling, payday lending, automated determinations of eligibility for credit, employment, educational institutions, or public assistance services.
- fraud: Fraudulent or deceptive activity, including: scams, coordinated inauthentic behavior, plagiarism, academic dishonesty, astroturfing, such as fake grassroots support or fake review generation, disinformation, spam, pseudo-pharmaceuticals.
- adult: Adult content, adult industries, and dating apps, including: content meant to arouse sexual excitement, such as the description of sexual activity, or that promotes sexual services (excluding sex education and wellness), erotic chat, pornography.
- political: Political campaigning or lobbying, by: generating high volumes of campaign materials, generating campaign materials personalized to or targeted at specific demographics, building conversational or interactive systems such as chatbots that provide information about campaigns or engage in political advocacy or lobbying, building products for political campaigning or lobbying purposes.
- privacy: Activity that violates people's privacy, including: tracking or monitoring an individual without their consent, facial recognition of private individuals, classifying individuals based on protected characteristics, using biometrics for identification or assessment, unlawful collection or disclosure of personal identifiable information or educational, financial, or other protected records.
- unqualified law: Engaging in the unauthorized practice of law, or offering tailored legal advice without a qualified person reviewing the information.
- unqualified financial: Offering tailored financial advice without a qualified person reviewing the information.
- unqualified health: Telling someone that they have or do not have a certain health condition, or providing instructions on how to cure or treat a health condition.
Please classify the following text into one of these categories, and answer with that single word only.
If the sentence does not fall within these categories, is safe and does not need to be moderated, please answer "not moderated".
Text: 개인정보는 무조건 팔아야지"""
}]

# Tokenize the prompt
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

In [ ]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))